In [1]:
import csv
import email
import email.header as he
import imaplib
import re
import pymysql
from bs4 import BeautifulSoup as bso

db = pymysql.connect(host="localhost",user="root",passwd="",database="scrapper")
#db = pymysql.connect(host="35.222.64.127",user="root",password="Some@Moreyeahs",db="mydb",port=36520)
cursor = db.cursor()

id='livecontentrepo@gmail.com'
pw='Nica345!!'

n,c=0,0

In [2]:
def insert_data(db_para):
    for i in db_para:
        try:
            sql = "INSERT INTO scrapper(sender,subject,urlzoom,date,evtime,mbody) VALUES (%s,%s,%s,%s,%s,%s)"
            val=(db_para[i]["sender"],db_para[i]["subject"],db_para[i]["zoomLink"],db_para[i]["date"],db_para[i]["evtime"],db_para[i]["mbody"])

            cursor.execute(sql,val)
            db.commit()
            global c
            c+=1
            print(c,"record inserted.")
        except:
            continue
        finally:
            db_para[i].clear()


In [3]:
def read_email_from_gmail():
    e=0

    try:
        mail = imaplib.IMAP4_SSL('imap.gmail.com')

    except(SystemError):
        pass


    mail.login(id,pw)
    mail.select('Confirmation')

    _,searched_data= mail.search(None,"FROM Eventbrite SINCE 18-MAR-2021")
    #_,searched_data= mail.search(None,"ON {}".format('14-Dec-2020'))
    mail_ids = searched_data[0]

    id_list = mail_ids.split()
    first_email_id = int(id_list[0])
    latest_email_id = int(id_list[-1])+1

    p=latest_email_id-first_email_id
    print('total mail=',p)
    db_data={}
    e=0
    match=""
    for i in range(first_email_id,latest_email_id):
        e+=1
        _, data = mail.fetch(str(i), '(RFC822)')
        b=data[0][1]
        email_data={}
        email_message = email.message_from_bytes(b)

        email_subject = str(email_message['subject'])
        email_subject= he.decode_header(email_subject)[0][0]
        if isinstance(email_subject,bytes):
            email_subject=email_subject.decode('latin1')

        email_from = email_message['from']
        time = email_message['date']
        print(time)

        for part in email_message.walk():
            if part.get_content_type() == "text/html":
                html_body = part.get_payload(decode=True)
                email_data['body'] = html_body.decode('latin-1')
                soup1=bso(email_data['body'],'lxml')
        fullbody=""
        try:
            for i in soup1.body.strings:
                fullbody+=i
        except (AttributeError, KeyError):
            continue
        fullbody=' '.join(fullbody.split())
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',fullbody)
        find_zoom = ['zoom.us/j/']
        zoomLink = ""
        t=""
        for word in find_zoom:
            for phrase in url:
                if word.lower() in phrase.lower():
                    zoomLink+=phrase+"\n"
        global n
        n+=1


        if re.findall('\d{2}\d+', zoomLink):
            match=re.findall('\d{2}\d+', zoomLink)

        else:
            match="None"



        db_data[str(n)]={"sender":email_from,"subject":email_subject,"zoomLink":zoomLink,"date":time,"evtime":t,"mbody":fullbody}
        main_body=""
        main_body+=soup1.prettify()



        #field = ['mailf','subject','link','time','to','body']
        myData = [[email_from, email_subject, zoomLink, time, t, fullbody,match,main_body]]
        # myFile = open('newscrap.csv', 'w', encoding="utf-8")
        file = "march18.csv"
        with open(file,'a',newline='', encoding="utf-8") as f:
            cw = csv.writer(f)
            for i in myData:
                cw.writerow(i)
        if n==20:
            insert_data(db_data)
            n=0
        elif(e==p):
            insert_data(db_data)
            break
        else:
            pass


In [4]:
read_email_from_gmail()
db.rollback()
db.close()

total mail= 1359
Thu, 18 Mar 2021 06:00:08 -0000
Thu, 18 Mar 2021 06:00:27 -0000
Thu, 18 Mar 2021 06:01:02 -0000
Thu, 18 Mar 2021 06:01:05 -0000
Thu, 18 Mar 2021 06:00:59 -0000
Thu, 18 Mar 2021 06:01:19 -0000
Thu, 18 Mar 2021 06:01:40 -0000
Thu, 18 Mar 2021 06:01:35 -0000
Thu, 18 Mar 2021 06:01:33 -0000
Thu, 18 Mar 2021 06:01:49 -0000
Thu, 18 Mar 2021 06:01:33 -0000
Thu, 18 Mar 2021 06:02:09 -0000
Thu, 18 Mar 2021 06:02:04 -0000
Thu, 18 Mar 2021 06:02:15 -0000
Thu, 18 Mar 2021 06:02:32 -0000
Thu, 18 Mar 2021 06:02:46 -0000
Thu, 18 Mar 2021 06:02:54 -0000
Thu, 18 Mar 2021 06:03:23 -0000
Thu, 18 Mar 2021 06:03:27 -0000
Thu, 18 Mar 2021 06:03:19 -0000
Thu, 18 Mar 2021 06:03:34 -0000
Thu, 18 Mar 2021 06:03:55 -0000
Thu, 18 Mar 2021 06:04:02 -0000
Thu, 18 Mar 2021 06:04:13 -0000
Thu, 18 Mar 2021 06:04:32 -0000
Thu, 18 Mar 2021 06:04:55 -0000
Thu, 18 Mar 2021 06:05:07 -0000
Thu, 18 Mar 2021 06:05:10 -0000
Thu, 18 Mar 2021 06:05:43 -0000
Thu, 18 Mar 2021 06:05:49 -0000
Thu, 18 Mar 2021 06:06:

Thu, 18 Mar 2021 06:53:56 -0000
Thu, 18 Mar 2021 06:54:26 -0000
Thu, 18 Mar 2021 06:54:37 -0000
Thu, 18 Mar 2021 06:54:32 -0000
Thu, 18 Mar 2021 06:55:08 -0000
Thu, 18 Mar 2021 06:55:10 -0000
Thu, 18 Mar 2021 06:55:12 -0000
Thu, 18 Mar 2021 06:55:45 -0000
Thu, 18 Mar 2021 06:55:57 -0000
Thu, 18 Mar 2021 06:56:22 -0000
Thu, 18 Mar 2021 06:56:26 -0000
Thu, 18 Mar 2021 06:56:40 -0000
Thu, 18 Mar 2021 06:56:58 -0000
Thu, 18 Mar 2021 06:57:25 -0000
Thu, 18 Mar 2021 06:57:22 -0000
Thu, 18 Mar 2021 06:57:33 -0000
Thu, 18 Mar 2021 06:57:42 -0000
Thu, 18 Mar 2021 06:58:03 -0000
Thu, 18 Mar 2021 06:58:08 -0000
Thu, 18 Mar 2021 06:58:13 -0000
Thu, 18 Mar 2021 06:58:30 -0000
Thu, 18 Mar 2021 06:58:45 -0000
Thu, 18 Mar 2021 06:58:54 -0000
Thu, 18 Mar 2021 06:59:21 -0000
Thu, 18 Mar 2021 06:59:33 -0000
Thu, 18 Mar 2021 06:59:48 -0000
Thu, 18 Mar 2021 06:59:52 -0000
Thu, 18 Mar 2021 06:59:57 -0000
Thu, 18 Mar 2021 07:00:30 -0000
Thu, 18 Mar 2021 07:00:33 -0000
Thu, 18 Mar 2021 07:01:05 -0000
Thu, 18 

Thu, 18 Mar 2021 07:53:22 -0000
Thu, 18 Mar 2021 07:53:47 -0000
Thu, 18 Mar 2021 07:53:52 -0000
Thu, 18 Mar 2021 07:54:08 -0000
Thu, 18 Mar 2021 07:54:32 -0000
Thu, 18 Mar 2021 07:54:43 -0000
Thu, 18 Mar 2021 07:54:56 -0000
Thu, 18 Mar 2021 07:55:19 -0000
Thu, 18 Mar 2021 07:55:21 -0000
Thu, 18 Mar 2021 07:55:27 -0000
Thu, 18 Mar 2021 07:55:38 -0000
Thu, 18 Mar 2021 07:56:02 -0000
Thu, 18 Mar 2021 07:56:43 -0000
Thu, 18 Mar 2021 07:56:55 -0000
Thu, 18 Mar 2021 07:57:23 -0000
Thu, 18 Mar 2021 07:57:25 -0000
Thu, 18 Mar 2021 07:57:54 -0000
Thu, 18 Mar 2021 07:58:15 -0000
Thu, 18 Mar 2021 07:58:31 -0000
Thu, 18 Mar 2021 07:58:44 -0000
Thu, 18 Mar 2021 07:58:56 -0000
Thu, 18 Mar 2021 07:59:15 -0000
Thu, 18 Mar 2021 07:59:29 -0000
Thu, 18 Mar 2021 07:59:37 -0000
Thu, 18 Mar 2021 07:59:58 -0000
Thu, 18 Mar 2021 08:00:10 -0000
Thu, 18 Mar 2021 08:00:13 -0000
Thu, 18 Mar 2021 08:00:42 -0000
Thu, 18 Mar 2021 08:00:41 -0000
Thu, 18 Mar 2021 08:00:56 -0000
Thu, 18 Mar 2021 08:01:02 -0000
Thu, 18 

Thu, 18 Mar 2021 08:57:31 -0000
Thu, 18 Mar 2021 08:57:32 -0000
Thu, 18 Mar 2021 08:58:20 -0000
Thu, 18 Mar 2021 08:58:36 -0000
Thu, 18 Mar 2021 08:58:49 -0000
Thu, 18 Mar 2021 08:59:01 -0000
Thu, 18 Mar 2021 08:59:15 -0000
Thu, 18 Mar 2021 08:59:16 -0000
Thu, 18 Mar 2021 08:59:41 -0000
Thu, 18 Mar 2021 09:00:06 -0000
Thu, 18 Mar 2021 09:00:08 -0000
Thu, 18 Mar 2021 09:00:53 -0000
Thu, 18 Mar 2021 09:00:57 -0000
Thu, 18 Mar 2021 09:01:05 -0000
Thu, 18 Mar 2021 09:01:10 -0000
Thu, 18 Mar 2021 09:01:35 -0000
Thu, 18 Mar 2021 09:01:38 -0000
Thu, 18 Mar 2021 09:02:05 -0000
Thu, 18 Mar 2021 09:02:17 -0000
Thu, 18 Mar 2021 09:02:47 -0000
Thu, 18 Mar 2021 09:02:54 -0000
Thu, 18 Mar 2021 09:02:54 -0000
Thu, 18 Mar 2021 09:03:32 -0000
Thu, 18 Mar 2021 09:03:36 -0000
Thu, 18 Mar 2021 09:03:38 -0000
Thu, 18 Mar 2021 09:03:38 -0000
Thu, 18 Mar 2021 09:04:12 -0000
Thu, 18 Mar 2021 09:04:12 -0000
Thu, 18 Mar 2021 09:04:15 -0000
Thu, 18 Mar 2021 09:04:22 -0000
Thu, 18 Mar 2021 09:04:47 -0000
Thu, 18 

Thu, 18 Mar 2021 10:00:09 -0000
Thu, 18 Mar 2021 10:00:15 -0000
Thu, 18 Mar 2021 10:00:35 -0000
Thu, 18 Mar 2021 10:00:45 -0000
Thu, 18 Mar 2021 10:00:50 -0000
Thu, 18 Mar 2021 10:01:17 -0000
Thu, 18 Mar 2021 10:01:28 -0000
Thu, 18 Mar 2021 10:01:33 -0000
Thu, 18 Mar 2021 10:01:55 -0000
Thu, 18 Mar 2021 10:02:37 -0000
Thu, 18 Mar 2021 10:02:41 -0000
Thu, 18 Mar 2021 10:03:18 -0000
Thu, 18 Mar 2021 10:04:06 -0000
Thu, 18 Mar 2021 10:03:42 -0000
Thu, 18 Mar 2021 10:03:45 -0000
Thu, 18 Mar 2021 10:04:24 -0000
Thu, 18 Mar 2021 10:05:11 -0000
Thu, 18 Mar 2021 10:05:37 -0000
Thu, 18 Mar 2021 10:05:55 -0000
Thu, 18 Mar 2021 10:06:06 -0000
Thu, 18 Mar 2021 10:06:15 -0000
Thu, 18 Mar 2021 10:06:22 -0000
Thu, 18 Mar 2021 10:07:17 -0000
Thu, 18 Mar 2021 10:07:28 -0000
Thu, 18 Mar 2021 10:07:47 -0000
Thu, 18 Mar 2021 10:07:59 -0000
Thu, 18 Mar 2021 10:08:00 -0000
Thu, 18 Mar 2021 10:08:38 -0000
Thu, 18 Mar 2021 10:08:49 -0000
Thu, 18 Mar 2021 10:09:12 -0000
Thu, 18 Mar 2021 10:09:33 -0000
Thu, 18 

Thu, 18 Mar 2021 11:19:45 -0000
Thu, 18 Mar 2021 11:20:03 -0000
Thu, 18 Mar 2021 11:20:27 -0000
Thu, 18 Mar 2021 11:20:37 -0000
Thu, 18 Mar 2021 11:20:44 -0000
Thu, 18 Mar 2021 11:20:45 -0000
Thu, 18 Mar 2021 11:21:13 -0000
Thu, 18 Mar 2021 11:21:13 -0000
Thu, 18 Mar 2021 11:22:22 -0000
Thu, 18 Mar 2021 11:22:51 -0000
Thu, 18 Mar 2021 11:23:00 -0000
Thu, 18 Mar 2021 11:23:17 -0000
Thu, 18 Mar 2021 11:23:22 -0000
Thu, 18 Mar 2021 11:23:27 -0000
Thu, 18 Mar 2021 11:24:13 -0000
Thu, 18 Mar 2021 11:24:32 -0000
Thu, 18 Mar 2021 11:25:19 -0000
Thu, 18 Mar 2021 11:25:32 -0000
Thu, 18 Mar 2021 11:25:40 -0000
Thu, 18 Mar 2021 11:26:27 -0000
Thu, 18 Mar 2021 11:26:52 -0000
Thu, 18 Mar 2021 11:27:51 -0000
Thu, 18 Mar 2021 11:27:53 -0000
Thu, 18 Mar 2021 11:28:30 -0000
Thu, 18 Mar 2021 11:28:40 -0000
Thu, 18 Mar 2021 11:29:33 -0000
Thu, 18 Mar 2021 11:30:09 -0000
Thu, 18 Mar 2021 11:30:48 -0000
Thu, 18 Mar 2021 11:30:56 -0000
Thu, 18 Mar 2021 11:31:50 -0000
Thu, 18 Mar 2021 11:32:26 -0000
Thu, 18 

In [ ]:
final_df = pd.read_csv("march02.csv")
final_df.columns=['a','b','c','d','e','f','g','h']
final_df = final_df[final_df['c'].notnull()]
final_df.reset_index(inplace=True)
final_df

In [ ]:
import re
u = "[A-Z][a-z]+,\s?\d*\s[A-Z][a-z]+\s?\d*,?\s\d\d\d\d\s?[a-z]*?\s?\d*:?\d*?\s?[A-Z]*?\s[a-z]+\s\d*:\d*\s?[A-Z]*?\s\(?[A-Z]+\)?"
pattern = re.compile(u)
for i,j in enumerate(final_df["f"]):
    try:
        found=pattern.findall(j)
        found_str=""
        found_str=found_str.join(found)
    except:
        pass
    if found==[]:
        pass
    else:
        print("hey")
        final_df.at[i,"start_time"]=found_str

In [ ]:
final_df["start_time"]=final_df["start_time"].str.replace("at",'',regex=True)

In [ ]:
df=pd.read_csv()